In [ ]:
import random

def query(x):
    return -1 * (x - 7)**2 + 49

def HillClimbing(N):
    x = random.randint(0, N - 1)
    while True:
        if x + 1 <= N and query(x) < query(x + 1):
            x += 1
        elif x - 1 >= 0 and query(x) < query(x - 1):
            x -= 1
        else:
            return x

print(HillClimbing(14))


7


In [ ]:

##QUESTION2:
import random
task_times = [5, 8, 4, 7, 6, 3, 9]
facility_capacities = [24, 30, 28]
cost_matrix = [
    [10, 12, 9],
    [15, 14, 16],
    [8,  9,  7],
    [12, 10, 13],
    [14, 13, 12],
    [9,  8,  10],
    [11, 12, 13],
]
def initialize_population(size):
  population = []
  for i in range(size):
    chromosome=[]
    for i in range(7):
      chromosome.append(random.randint(0,2))
    population.append(chromosome)
  return population
#print(initialize_population(6))

def calculate_fitness(population,size):
  fitness=[]
  cost=0
  check=[0,0,0]
  for i in range(size):
    cost=0
    for j in range(7):
      a=cost_matrix[j][population[i][j]]*task_times[j]
      cost+=a
      check[population[i][j]]+=task_times[j]
    for ind in range(3):
      if check[ind]>facility_capacities[ind]:
        cost+=500
    check=[0,0,0]
    fitness.append(cost)
  return fitness

def select_parents(population, fitness, size):
  maxi=max(fitness)
  parents=[]
  prob=[(maxi+10)-fit for fit in fitness]
  summ=sum(prob)
  prob_cumsum=[prob[0]]
  for i in range(1,size):
    prob_cumsum.append(prob_cumsum[i-1]+prob[i])
  for i in range(int(0.8 * size)):
        x = random.randint(0, summ)
        for i in range(size):
            if x <= prob_cumsum[i]:
                parents.append(population[i])

  return parents

def cross_over(parents,size):
  children=[]
  for i in range(size):
    x=random.randint(0,5)
    child=parents[i][:x]
    child+=parents[(i+1)%size][x:]
    children.append(child)
  return children

def mutate(chromosome, mutation_rate=0.2):

    if random.random() < mutation_rate:

        idx1, idx2 = random.sample(range(len(chromosome)), 2)


        chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]

    return chromosome

def genetic_algorithm(population_size, num_generations):
    population = initialize_population(population_size)
    for generation in range(num_generations):
        fitness = calculate_fitness(population, population_size)
        parents = select_parents(population, fitness, population_size)
        children = cross_over(parents, population_size)
        for i in range(population_size):
            children[i] = mutate(children[i])
        population = children

        best_fitness = min(fitness)
    best_chromosome = population[fitness.index(best_fitness)]
    return best_chromosome, best_fitness

print(genetic_algorithm(6,20))
print(genetic_algorithm(6,30))

([2, 0, 0, 2, 1, 0, 0], 492)
([1, 2, 1, 1, 2, 2, 0], 477)


In [7]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [11]:
from queue import Queue
from copy import deepcopy
import time

# Initial Sudoku Board
sudoku = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

def domains(board):
    dom = {}
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                dom[(i, j)] = list(range(1, 10))
            else:
                dom[(i, j)] = [board[i][j]]
    return dom

def neighbours(r, c):
    nbours = set()
    for i in range(9):
        if i != r:
            nbours.add((i, c))
        if i != c:
            nbours.add((r, i))
    for i in range(r // 3 * 3, r // 3 * 3 + 3):
        for j in range(c // 3 * 3, c // 3 * 3 + 3):
            if (i, j) != (r, c):
                nbours.add((i, j))
    return list(nbours)

def AC3(domain, neighbours_func):
    queue = Queue()
    for i in range(9):
        for j in range(9):
            for (a, b) in neighbours_func(i, j):
                queue.put(((i, j), (a, b)))
    while not queue.empty():
        (xi, xj) = queue.get()
        val = revise(xi, xj, domain)
        if val == -1:
            continue
        if val == 0:
            return False
        if val == 1:
            for (a, b) in neighbours_func(xi[0], xi[1]):
                queue.put(((a, b), xi))
    return True

def revise(xi, xj, domain):
    revised = False
    if len(domain[xj]) == 1:
        val = domain[xj][0]
        if val in domain[xi]:
            domain[xi] = [x for x in domain[xi] if x != val]
            revised = True
    if len(domain[xi]) == 0:
        return 0
    return 1 if revised else -1

def solve_with_domain(board, domain):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                for val in domain[(i, j)]:
                    conflict = False
                    for (a, b) in neighbours(i, j):
                        if board[a][b] == val:
                            conflict = True
                            break
                    if not conflict:
                        new_board = deepcopy(board)
                        new_board[i][j] = val
                        new_domain = deepcopy(domain)
                        new_domain[(i, j)] = [val]
                        if AC3(new_domain, neighbours):
                            result = solve_with_domain(new_board, new_domain)
                            if result:
                                return result
                return False
    return board

def solve_with_ac3_and_backtracking():
    board = deepcopy(sudoku)
    domain = domains(board)
    if AC3(domain, neighbours):
        result = solve_with_domain(board, domain)
        if result:
            for row in result:
                print(" ".join(map(str, row)))
        else:
            print("No solution exists.")
    else:
        print("Inconsistent constraints.")

# Timing the solver
if __name__ == "__main__":
    start = time.time()
    solve_with_ac3_and_backtracking()
    end = time.time()
    print(f"\nAC3 + Backtracking Solver took {end - start:.4f} seconds")


5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9

AC3 + Backtracking Solver took 0.3450 seconds


In [12]:
import time
from ortools.sat.python import cp_model

# Sudoku puzzle (0 means empty)
sudoku = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

def solve_ortools_sudoku():
    model = cp_model.CpModel()
    cells = []

    for i in range(9):
        row = []
        for j in range(9):
            if sudoku[i][j] == 0:
                row.append(model.NewIntVar(1, 9, f'cell_{i}_{j}'))
            else:
                row.append(model.NewConstant(sudoku[i][j]))
        cells.append(row)

    for i in range(9):
        model.AddAllDifferent(cells[i])
    for j in range(9):
        model.AddAllDifferent([cells[i][j] for i in range(9)])
    for box_row in range(3):
        for box_col in range(3):
            box = [cells[box_row*3 + i][box_col*3 + j] for i in range(3) for j in range(3)]
            model.AddAllDifferent(box)

    solver = cp_model.CpSolver()
    start = time.time()
    status = solver.Solve(model)
    end = time.time()

    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
        for i in range(9):
            print(" ".join(str(solver.Value(cells[i][j])) for j in range(9)))
        print(f"\nTime taken: {end - start:.4f} seconds")
    else:
        print("No solution found.")

solve_ortools_sudoku()


5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9

Time taken: 0.0046 seconds


In [14]:
import time

class Sudoku:
    def __init__(self, grid):
        self.grid = grid
        self.possible_values = [[set(range(1, 10)) if cell == 0 else {cell} for cell in row] for row in grid]

    def is_complete(self):
        return all(len(self.possible_values[i][j]) == 1 for i in range(9) for j in range(9))

    def get_possible_values(self, row, col):
        return list(self.possible_values[row][col])

    def set_possible_values(self, row, col, values):
        self.possible_values[row][col] = set(values)

    def set_element(self, row, col, value):
        self.possible_values[row][col] = {value}

    def get_element(self, row, col):
        vals = self.possible_values[row][col]
        return next(iter(vals)) if len(vals) == 1 else 0

    def get_neighbors(self, row, col):
        neighbors = set()
        for i in range(9):
            if i != col:
                neighbors.add((row, i))
            if i != row:
                neighbors.add((i, col))
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if (i, j) != (row, col):
                    neighbors.add((i, j))
        return list(neighbors)

    def deep_copy(self):
        new_grid = [[self.get_element(i, j) for j in range(9)] for i in range(9)]
        return Sudoku(new_grid)

    def display(self):
        for i in range(9):
            row = [str(self.get_element(i, j)) if self.get_element(i, j) != 0 else '.' for j in range(9)]
            print(' '.join(row))


class SudokuCSP:
    def __init__(self, sudoku):
        self.sudoku = sudoku

    def get_initial_sudoku(self):
        return self.sudoku.deep_copy()

    def select_unassigned_variable(self, assignment):
        # MRV: Select the variable with the fewest possible values
        min_domain_size = float('inf')
        selected_variable = None

        for i in range(9):
            for j in range(9):
                if len(assignment.get_possible_values(i, j)) > 1:  # Only consider unassigned cells
                    domain_size = len(assignment.get_possible_values(i, j))
                    if domain_size < min_domain_size:
                        min_domain_size = domain_size
                        selected_variable = (i, j)

        return selected_variable

    def order_domain_values(self, var, assignment):
        return assignment.get_possible_values(var[0], var[1])

    def is_consistent(self, var, value, assignment):
        for neighbor in assignment.get_neighbors(var[0], var[1]):
            neighbor_vals = assignment.get_possible_values(neighbor[0], neighbor[1])
            if len(neighbor_vals) == 1 and neighbor_vals[0] == value:
                return False
        return True


def backtracking_search(csp):
    assignment = csp.get_initial_sudoku()
    return recursive_backtracking(assignment, csp)

def recursive_backtracking(assignment, csp):
    if assignment.is_complete():
        return assignment

    var = csp.select_unassigned_variable(assignment)
    if var is None:
        return False

    for value in csp.order_domain_values(var, assignment):
        if csp.is_consistent(var, value, assignment):
            assignment_copy = assignment.deep_copy()
            assignment_copy.set_element(var[0], var[1], value)

            result = recursive_backtracking(assignment_copy, csp)
            if result:
                return result

    return False

def arc_consistency_3(sudoku, arcs=None):
    if arcs is None:
        arcs = []
        for row in range(9):
            for col in range(9):
                neighbors = sudoku.get_neighbors(row, col)
                for neighbor in neighbors:
                    arcs.append(((row, col), neighbor))

    while arcs:
        (xi, xj) = arcs.pop(0)
        if revise(sudoku, xi, xj):
            if len(sudoku.get_possible_values(xi[0], xi[1])) == 0:
                return False
            neighbors = sudoku.get_neighbors(xi[0], xi[1])
            neighbors.remove(xj)
            for xk in neighbors:
                arcs.append((xk, xi))
    return True

def revise(sudoku, xi, xj):
    revised = False
    xi_domain = sudoku.get_possible_values(xi[0], xi[1])
    xj_domain = sudoku.get_possible_values(xj[0], xj[1])

    for x in xi_domain[:]:
        if len(xj_domain) == 1 and xj_domain[0] == x:
            xi_domain.remove(x)
            revised = True

    if revised:
        sudoku.set_possible_values(xi[0], xi[1], xi_domain)

    return revised

# main.py
if __name__ == "__main__":
    puzzle = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    # Start the timer
    start_time = time.time()

    sudoku = Sudoku(puzzle)
    arc_consistency_3(sudoku)
    csp = SudokuCSP(sudoku)
    solution = backtracking_search(csp)

    # End the timer
    end_time = time.time()

    if solution:
        solution.display()
    else:
        print("No solution found.")

    # Print execution time
    print(f"Execution time: {end_time - start_time:.4f} seconds")


5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
Execution time: 0.0175 seconds


To Improve time on my original sudoku:


1. **Use MRV** and **Degree Heuristic** for variable selection.
2. Apply **Arc-Consistency (AC-3)** and **Forward Checking** to prune invalid values.
3. Preprocess with **sets** for rows, columns, and subgrids.
4. Optimize backtracking by pruning early and using **Early Exit**.
5. Use **stack-based backtracking** and **memoization** to avoid redundancy.
6. Consider **parallelization** or **Dancing Links** for advanced speed.